# Autoencoders NLP


In [1]:
import pandas as pd
import numpy as np
import warnings
import requests
import json
import re

from nltk.corpus import stopwords 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from gensim.models import word2vec
from sklearn import svm

import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


## 1. Cargando Datos

In [3]:
dataset = pd.read_excel('dataset_valoration.xlsx') # https://github.com/AnthonyWainer/AutoencoderNLP/blob/master/dataset/dataset_valoration.xlsx?raw=true

display(dataset.sort_index().head(5))
print('\n Hay {} observaciones con {} características'.format(*dataset.shape))

FileNotFoundError: ignored

## 2. Limpieza básica de Datos Textuales

In [6]:
dataset.loc[:, ('clean_comment')] = dataset.comment.apply(processing_words)

dataset = dataset.drop_duplicates(subset='clean_comment', keep="last")

dataset.sort_index(inplace=True)

display(dataset.sort_index().head(5))
print('\n Hay {} observaciones con {} características'.format(*dataset.shape))

,valoration,comment,clean_comment
0,0,totalmente satisfecho x q resolvieron mi cons...,totalmente satisfecho resolvieron consulta ate...
4,0,por que me atendieron muy amablemente con buen...,atendieron amablemente buena actitud plataform...
5,0,me atendieron bien con mucha atencion,atendieron bien mucha atencion
6,0,la atencion de las señoritas de la entrada fue...,atencion señoritas entrada buena momento supie...
7,0,porque hay varios cajerosas es una agencia rel...,varios cajerosas agencia relativamente grande



 Hay 7391 observaciones con 3 características


In [7]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

!pip install sentencepiece

In [8]:
import tensorflow_hub as hub
import tokenization

module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'

bert_layer = hub.KerasLayer(module_url, trainable=True)

In [89]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [134]:
dataset.loc[:, ('clean_comment_list')] = dataset.clean_comment.apply(lambda comment: comment.split())

display(dataset.sort_index().head(5))
print('\n Hay {} observaciones con {} características'.format(*dataset.shape))

,valoration,comment,clean_comment,vector_bert,clean_comment_list
0,0,totalmente satisfecho x q resolvieron mi cons...,totalmente satisfecho resolvieron consulta ate...,"([[101, 2561, 3672, 2063, 102, 0, 0, 0, 0, 0, ...","[totalmente, satisfecho, resolvieron, consulta..."
4,0,por que me atendieron muy amablemente con buen...,atendieron amablemente buena actitud plataform...,"([[101, 8823, 16089, 26534, 102, 0, 0, 0, 0, 0...","[atendieron, amablemente, buena, actitud, plat..."
5,0,me atendieron bien con mucha atencion,atendieron bien mucha atencion,"([[101, 8823, 16089, 26534, 102, 0, 0, 0, 0, 0...","[atendieron, bien, mucha, atencion]"
6,0,la atencion de las señoritas de la entrada fue...,atencion señoritas entrada buena momento supie...,"([[101, 8823, 12273, 3258, 102, 0, 0, 0, 0, 0,...","[atencion, señoritas, entrada, buena, momento,..."
7,0,porque hay varios cajerosas es una agencia rel...,varios cajerosas agencia relativamente grande,"([[101, 13075, 10735, 102, 0, 0, 0, 0, 0, 0, 0...","[varios, cajerosas, agencia, relativamente, gr..."



 Hay 7391 observaciones con 5 características


In [113]:
max_len = 150

dataset.loc[:, ('vector_bert')] = dataset.clean_comment.apply(lambda comment: bert_encode(comment.split(), tokenizer, max_len = max_len))

display(dataset.sort_index().head(5))
print('\n Hay {} observaciones con {} características'.format(*dataset.shape))

,valoration,comment,clean_comment,vector_bert
0,0,totalmente satisfecho x q resolvieron mi cons...,totalmente satisfecho resolvieron consulta ate...,"([[101, 2561, 3672, 2063, 102, 0, 0, 0, 0, 0, ..."
4,0,por que me atendieron muy amablemente con buen...,atendieron amablemente buena actitud plataform...,"([[101, 8823, 16089, 26534, 102, 0, 0, 0, 0, 0..."
5,0,me atendieron bien con mucha atencion,atendieron bien mucha atencion,"([[101, 8823, 16089, 26534, 102, 0, 0, 0, 0, 0..."
6,0,la atencion de las señoritas de la entrada fue...,atencion señoritas entrada buena momento supie...,"([[101, 8823, 12273, 3258, 102, 0, 0, 0, 0, 0,..."
7,0,porque hay varios cajerosas es una agencia rel...,varios cajerosas agencia relativamente grande,"([[101, 13075, 10735, 102, 0, 0, 0, 0, 0, 0, 0..."



 Hay 7391 observaciones con 4 características


## 3. TF - TF-idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
data = datos['Limpio'].values.tolist()

In [ ]:
vectorizer = TfidfVectorizer(max_features=100)
FV = vectorizer.fit_transform(data)
FV = FV.toarray()

In [ ]:
vectorizer.get_feature_names()

['ademas',
 'agencia',
 'amabilidad',
 'amable',
 'amables',
 'atencion',
 'atender',
 'atendido',
 'atendieron',
 'atendio',
 'atienden',
 'banca',
 'banco',
 'bien',
 'buen',
 'buena',
 'cajero',
 'cajeros',
 'cliente',
 'clientes',
 'cola',
 'colas',
 'como',
 'cordial',
 'credito',
 'cual',
 'cuando',
 'cuenta',
 'demora',
 'dieron',
 'dijeron',
 'dijo',
 'dinero',
 'dudas',
 'eficiente',
 'espera',
 'esperar',
 'esta',
 'estaba',
 'excelente',
 'exclusiva',
 'fueron',
 'gente',
 'gracias',
 'habia',
 'hacer',
 'hasta',
 'hora',
 'informacion',
 'luego',
 'manera',
 'minutos',
 'momento',
 'mucha',
 'mucho',
 'nada',
 'operacion',
 'otra',
 'otro',
 'pagar',
 'pago',
 'para',
 'parte',
 'pero',
 'persona',
 'personal',
 'personas',
 'pesar',
 'plataforma',
 'poco',
 'porque',
 'problema',
 'pude',
 'rapida',
 'rapidez',
 'rapido',
 'realizar',
 'servicio',
 'señorita',
 'siempre',
 'sistema',
 'sobre',
 'soles',
 'solo',
 'solucion',
 'srta',
 'tarjeta',
 'telefono',
 'tener',
 'te

In [ ]:
datos.head()

,EVALUACION,Comentariomin,Limpio
0,1,totalmente satisfecho x q resolvieron mi cons...,totalmente satisfecho resolvieron consulta ate...
1,1,la atencion fue rapida y clara,atencion rapida clara
2,1,me atendieron bien,atendieron bien
3,1,fue muy rapido la atencion,rapido atencion
4,1,por que me atendieron muy amablemente con buen...,atendieron amablemente buena actitud plataform...


In [ ]:
datos['FV'] = FV.tolist()
datos.head()

,EVALUACION,Comentariomin,Limpio,FV
0,1,totalmente satisfecho x q resolvieron mi cons...,totalmente satisfecho resolvieron consulta ate...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.23851904737385277,..."
1,1,la atencion fue rapida y clara,atencion rapida clara,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.5339982472146613, ..."
2,1,me atendieron bien,atendieron bien,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.667..."
3,1,fue muy rapido la atencion,rapido atencion,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.4562837644991273, ..."
4,1,por que me atendieron muy amablemente con buen...,atendieron amablemente buena actitud plataform...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.448..."


# Separando Train/Test (80%/20%)

In [148]:
X = bert_encode(dataset.clean_comment.values, tokenizer, max_len = max_len)
Y = np.array(dataset.valoration.to_list())

In [181]:
X[0][1]

array([  101,  8823, 16089, 26534, 25933,  3468,  3672,  2063, 27493,
        2552,  4183,  6784, 19534, 14192,  2050,  6187, 20009,  2080,
         102,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [190]:
X[0].reshape(-1,1)

array([[ 101],
       [2561],
       [3672],
       ...,
       [   0],
       [   0],
       [   0]])

In [160]:
dat

Exception: ignored

In [ ]:
X, y = train_df.loc[:, train_df.columns != 'Survived'], train_df['Survived']

In [1]:
X[0].shape, X[1].shape,X[2].shape,

NameError: ignored

In [149]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.2,
                                                   random_state=10)

ValueError: ignored

In [116]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((5912, 3), (1479, 3), (5912,), (1479,))

In [192]:
import tensorflow as tf

def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    net = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(32, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = tf.keras.layers.Dense(2, activation='softmax')(net)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [126]:
train_labels = keras.utils.to_categorical(dataset.valoration.values, num_classes=2)

In [193]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 150)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 150)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]           

In [128]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

In [131]:
y_train = keras.utils.to_categorical(y_train, num_classes=2)

In [ ]:
train_history = model.fit(
    X, train_labels, 
    validation_split=0.2,
    epochs=3,
    callbacks=[checkpoint, earlystopping],
    batch_size=32,
    verbose=1
)

Epoch 1/3


KeyboardInterrupt: ignored

## 4. SVM

In [117]:
clasificador = svm.SVC()
clasificador.fit(X_train,y_train)

ValueError: ignored

In [ ]:
Y_pred = clasificador.predict(X_test)

In [ ]:
Y_pred

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
from sklearn.metrics import accuracy_score,classification_report

In [ ]:
accuracy_score(y_test,Y_pred)

0.8791327913279133

In [ ]:
print(classification_report(y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.78      0.76      0.77       495
           1       0.91      0.92      0.92      1350

    accuracy                           0.88      1845
   macro avg       0.85      0.84      0.84      1845
weighted avg       0.88      0.88      0.88      1845



In [ ]:
texto = "rapida"
fv = vectorizer.transform([texto]).toarray()
clasificador.predict(fv)

array([1])

In [ ]:
texto = "demora"
fv = vectorizer.transform([texto]).toarray()
clasificador.predict(fv)

array([0])